# Sweep Transfer Attack

9.29.21

#### Summary:
- Made each client (7 options) adversarial and perform transfer attacks throughout the system

In [2]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Relevant Libraries
Take it from the run_experiment.py folder

In [1]:
import sys
sys.path.append("../")

# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

### Generate Aggregator Pre-requisite
- Clients, Test Clients, Ensemble_Learner
- Follow through the code in run_experiment.py

In [2]:
args_ = Args()
args_.experiment = "cifar10"
args_.method = "local"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar10/local/'
args_.validation = False
args_.tune_steps = 0
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████| 80/80 [00:00<00:00, 262.20it/s]


===> Initializing clients..


100%|██████████| 80/80 [00:12<00:00,  6.18it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.297 | Train Acc: 10.806% |Test Loss: 2.299 | Test Acc: 10.238% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [3]:
# # Import weights for aggregator
# aggregator.load_state(args_.save_path)

# # This is where the models are stored -- one for each mixture --> learner.model for nn
# hypotheses = aggregator.global_learners_ensemble.learners

# # obtain the state dict for each of the weights 
# weights_h = []

# for h in hypotheses:
#     weights_h += [h.model.state_dict()]
    
# # Determine here the different weights of models we desire to compare
# model_weights = []

# model_weights += [(1,0,0),(0,1,0),(0,0,1)] #single mixture
# model_weights += [(0.5,0.5,0),(0.5,0,0.5),(0,0.5,0.5)] # half of 2 mixtures of 3
# model_weights += [(1/3,1/3,1/3)] # Equally balanced from 3 mixtures

# # Generate the weights to test on as linear combinations of the model_weights
# models_test = []

# for (w0,w1,w2) in model_weights:
#     # first make the model with empty weights
#     new_model = copy.deepcopy(hypotheses[0].model)
#     new_model.eval()
#     new_weight_dict = copy.deepcopy(weights_h[0])
#     for key in weights_h[0]:
#         new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
#     new_model.load_state_dict(new_weight_dict)
#     models_test += [new_model]

num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

setting = 'local'

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/FedAvg_adv/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
#     args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy")
    weights = np.load(args_.save_path+"train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/figperturb/fedem_avg_p0_1/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [4]:
aggregator.global_learners_ensemble.learners

### Generate Data 

In [5]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

In [6]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

### Set Up Transfer Attack Scenario
- Set up order of which attacks will take place -- keep the same transferer, but simply swap out the weights of the adversary and flush out the existing data points analysis was done on
- Make dictionaries beforehand recording all of the metrics

In [7]:
# Set Up Dictionaries -- list holds the adversary idx
logs_adv = []

for i in range(len(model_weights)):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities'] = None
    adv_dict['orig_target_hit'] = None
    adv_dict['adv_target_hit'] = None
    adv_dict['metric_variance'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['metric_ingrad'] = None
    logs_adv += [adv_dict]

In [8]:
# Make transferer and Assign model index
victim_idxs = [0,1,2,3,4,5,6]

t1 = Transferer(models_list=models_test, dataloader=dataloader)

for adv_idx in victim_idxs:
    print("id", adv_idx)
    # Perform Attack
    
    t1.generate_victims(victim_idxs)
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "ifsgm")
    t1.send_to_victims(victim_idxs)
    # t1.check_empirical_metrics(orig_flag = True)
    t1.check_empirical_metrics()
    
    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = t1.orig_acc_transfers
    logs_adv[adv_idx]['orig_similarities'] = t1.orig_similarities
    logs_adv[adv_idx]['adv_acc_transfers'] = t1.adv_acc_transfers
    logs_adv[adv_idx]['adv_similarities'] = t1.adv_similarities
    logs_adv[adv_idx]['orig_target_hit'] = t1.orig_target_hit
    logs_adv[adv_idx]['adv_target_hit'] = t1.adv_target_hit
    
    logs_adv[adv_idx]['metric_variance'] = t1.metric_variance
    logs_adv[adv_idx]['metric_alignment'] = t1.metric_alignment
    logs_adv[adv_idx]['metric_ingrad'] = t1.metric_ingrad
    logs_adv[adv_idx]['metric_loss'] = t1.metric_loss

id 0


RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous

In [10]:
type(t1.advNN.trained_network)

torchvision.models.mobilenet.MobileNetV2

### Print and Record Results
- Organize the results into a matrix and print them into an excel sheet
- Following Categories (sim_benign, sim_adv, target_adv,grad_alignment)

In [16]:
metrics = ['orig_similarities','adv_similarities','adv_target_hit','metric_alignment']

sim_benign = np.zeros([len(victim_idxs),len(victim_idxs)]) 
sim_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
target_adv = np.zeros([len(victim_idxs),len(victim_idxs)]) 
grad_align = np.zeros([len(victim_idxs),len(victim_idxs)]) 

In [23]:
for adv_idx in victim_idxs:
    for victim in victim_idxs:
        sim_benign[adv_idx,victim] = logs_adv[adv_idx][metrics[0]][victim].data.tolist()
        sim_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[1]][victim].data.tolist()
        target_adv[adv_idx,victim] = logs_adv[adv_idx][metrics[2]][victim].data.tolist()
        grad_align[adv_idx,victim] = logs_adv[adv_idx][metrics[3]][victim].data.tolist()

In [24]:
# Save to Excel file

## convert your array into a dataframe
sim_benign_df = pd.DataFrame(sim_benign)
sim_adv_df = pd.DataFrame(sim_adv)
target_adv_df = pd.DataFrame(target_adv)
grad_align_df = pd.DataFrame(grad_align)

## save to xlsx file

#filepath = 'my_excel_file.xlsx'

# df.to_excel(filepath, index=False)

In [25]:
sim_benign_df

,0,1,2,3,4,5,6
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [26]:
sim_adv_df

,0,1,2,3,4,5,6
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [27]:
target_adv_df

,0,1,2,3,4,5,6
0,0.792411,0.792411,0.792411,0.792411,0.792411,0.792411,0.792411
1,0.744444,0.744444,0.744444,0.744444,0.744444,0.744444,0.744444
2,0.784922,0.784922,0.784922,0.784922,0.784922,0.784922,0.784922
3,0.771242,0.771242,0.771242,0.771242,0.771242,0.771242,0.771242
4,0.758465,0.758465,0.758465,0.758465,0.758465,0.758465,0.758465
5,0.743304,0.743304,0.743304,0.743304,0.743304,0.743304,0.743304
6,0.735294,0.735294,0.735294,0.735294,0.735294,0.735294,0.735294


In [28]:
grad_align_df

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def avg_nondiag1(array2d):
    d1 = array2d.shape[0]
    d2 = array2d.shape[1]
    
    counter = 0
    val = 0

    v = []
    
    for i1 in range(d1):
        for i2 in range(d2):
            if i1 != i2:
                counter+=1
                v.append(array2d[i1,i2])
    
    return np.array(v)

z = avg_nondiag1(grad_align_df.to_numpy())
print(f'grad_allign {z.mean()} {z.std()}')

grad_allign 0.0 0.0


In [36]:
z = np.array([logs_adv[id]['metric_loss'].detach().cpu().numpy() for id in (victim_idxs)]).var(axis=0)

print(f'{np.mean(z)} +- {np.std(z)}')

6.1332879066467285 +- 3.925236701965332


In [33]:
[logs_adv[id]['metric_loss'] for id in (victim_idxs)]

[tensor([-14.6847,  -9.0861, -12.1590, -13.0355,  -5.5307, -13.0642, -11.6901,
          -9.6894,  -9.6512,  -9.2321,  -9.7385, -10.8485,  -6.6340,  -8.3732,
          -5.5682, -11.2091, -10.6277,  -9.3931, -14.4224,  -9.3846,  -8.0889,
          -6.4126, -11.4183,  -8.0432, -10.9291,  -9.7856, -11.1508,  -4.7621,
          -7.2178,  -8.8826, -10.5308,  -8.7114, -11.6912,  -5.5029,  -7.8941,
         -12.3183, -10.3462,  -8.0230, -10.1024,  -8.0224,  -6.9404,  -8.4764,
         -13.2745,  -8.2184,  -6.2232, -12.3560, -11.2209, -15.3736,  -7.3440,
         -10.0745, -11.9058, -12.4361,  -9.1927,  -7.6689, -10.1026,  -7.5964,
          -4.3444, -10.6786,  -9.0685,  -6.1704, -14.1429,  -8.9850,  -8.8894,
         -11.4450,  -6.7559,  -5.4971,  -6.0202,  -8.4138,  -9.4734,  -6.2315,
          -9.0226,  -9.0838,  -9.6282, -11.1408, -15.5767,  -8.0454, -12.8567,
         -12.7224,  -9.2411,  -6.4083,  -7.7427,  -9.5992,  -6.8571,  -9.2849,
         -13.4315,  -7.0483, -13.6009, -10.2901,  -7

In [32]:
for id in len(victim_idxs):
    print(id)

TypeError: 'int' object is not iterable